In [15]:
"""
Created on Mon May 19 17:30 2025

Prepare the file prepared by Pierre for all the small basins to contain zmin and zmax

@author: Clara Burgard
"""

'\nCreated on Mon May 19 17:30 2025\n\nPrepare the file prepared by Pierre for all the small basins to contain zmin and zmax\n\n@author: Clara Burgard\n'

In [14]:
import xarray as xr
from tqdm.notebook import tqdm
from datetime import date
import numpy as np

In [2]:
%matplotlib qt5

QStandardPaths: error creating runtime directory '/run/user/2784' (Permission denied)


In [30]:
inputpath_raw = '/data/cburgard/MELT_PARAM_NONRESOLVED/raw/'
inputpath_interim = '/data/cburgard/PREPARE_FORCING/PREPARE_CAVITY_MASKS/interim/'
inputpath_interim2 = '/data/cburgard/MELT_PARAM_NONRESOLVED/interim/'

In [7]:
file_Justine = xr.open_dataset(inputpath_raw + 'basins_for_param_v5_raster_extrapolate.nc')
BedMachine_mask_2km= xr.open_dataset(inputpath_interim + 'BedMachine_v3_aggregated2km_masks_only.nc')
BedMachine_vars_2km= xr.open_dataset(inputpath_interim + 'BedMachine_v3_aggregated2km_allvars_withcategories.nc')

In [8]:
# select ice_draft for the part of the cell covered by an ice shelf only
ice_draft = (BedMachine_vars_2km['thickness'] - BedMachine_vars_2km['surface']).sel(category=3).drop('category')

In [21]:
unique_masks = np.unique(file_Justine['Band1'])
file_Justine = file_Justine.assign_coords({'ID': unique_masks.astype(int)})

In [23]:
zmin_list = []
zmax_list = []
perc_01_list = []
perc_99_list = []
for kisf in tqdm(file_Justine['ID']):
    isf_mask = (file_Justine['Band1'] == kisf) & (BedMachine_mask_2km['isf_conc'] > 0)
    
    zmin = ice_draft.where(isf_mask).min()
    zmin_list.append(zmin.assign_coords({'ID': kisf}))
    zmax = ice_draft.where(isf_mask).max()
    zmax_list.append(zmax.assign_coords({'ID': kisf}))
    perc_01 = ice_draft.where(isf_mask).quantile(0.01)
    perc_01_list.append(perc_01.assign_coords({'ID': kisf}))
    perc_99 = ice_draft.where(isf_mask).quantile(0.99)
    perc_99_list.append(perc_99.assign_coords({'ID': kisf}))

zmin_all = xr.concat(zmin_list, dim='ID')
zmax_all = xr.concat(zmax_list, dim='ID')
perc_01_all = xr.concat(perc_01_list, dim='ID')
perc_99_all = xr.concat(perc_99_list, dim='ID')

  0%|          | 0/134 [00:00<?, ?it/s]

/home/cburgard/.local/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1556: RuntimeWarning: All-NaN slice encountered
  return function_base._ureduce(a,


In [25]:
file_Justine['z_min'] = zmin_all
file_Justine['z_min'].attrs['standard_name'] = 'most shallow point of the ice-shelf draft'
file_Justine['z_min'].attrs['units'] = 'm below sea level'

file_Justine['z_max'] = zmax_all
file_Justine['z_max'].attrs['standard_name'] = 'deepest point of the ice-shelf draft'
file_Justine['z_max'].attrs['units'] = 'm below sea level'

file_Justine['z_perc01'] = perc_01_all.drop('quantile')
file_Justine['z_perc01'].attrs['standard_name'] = '1st percentile over ice-shelf draft'
file_Justine['z_perc01'].attrs['units'] = 'm below sea level'

file_Justine['z_perc99'] = perc_99_all.drop('quantile')
file_Justine['z_perc99'].attrs['standard_name'] = '99th percentile over ice-shelf draft'
file_Justine['z_perc99'].attrs['units'] = 'm below sea level'

file_Justine['isf_conc'] = BedMachine_mask_2km['isf_conc']
file_Justine['isf_conc'].attrs['standard_name'] = 'Ice-shelf concentration in the grid cell between 0 and 1'

file_Justine = file_Justine.rename({'Band1': 'basin_extrap'})

In [27]:
file_Justine.attrs=dict(Source='Modified file based on basins_for_param_v5_raster_extrapolate.nc (individual basins delimited by J. Mouginot)',
                      Creator='C. Burgard and P. Mathiot ('+date.today().strftime("%b-%d-%Y")+')')

In [31]:
file_Justine.to_netcdf(inputpath_interim2 + 'basins_for_param_v5_raster_extrapolate_with_zmin_zmax_isfconc.nc')

In [5]:
file_Justine = xr.open_dataset(inputpath_interim + 'Mask_Iceshelf_IMBIE2_v2_with_zmin_zmax_isfconc.nc')

In [6]:
file_Justine

<xarray.Dataset>
Dimensions:                  (ID: 133, x: 3041, y: 3041)
Coordinates:
  * ID                       (ID) int64 1 2 3 4 5 6 ... 128 129 130 131 132 133
  * x                        (x) int32 -3040000 -3038000 ... 3038000 3040000
  * y                        (y) int32 -3040000 -3038000 ... 3038000 3040000
    lat                      (y, x) float64 ...
    lon                      (y, x) float64 ...
    quantile                 float64 ...
Data variables: (12/21)
    NAME                     (ID) object ...
    Regions                  (ID) object ...
    Subregions               (ID) object ...
    MeltRignot               (ID) float64 ...
    UncertaintiesRignot      (ID) float64 ...
    Iceshelf                 (y, x) float32 ...
    ...                       ...
    UncertaintiesDavison     (ID) float64 ...
    z_min                    (ID) float64 ...
    z_max                    (ID) float64 ...
    z_perc01                 (ID) float64 ...
    z_perc99                 (ID) float64 ...
    isf_conc                 (y, x) float64 ...

In [ ]:
for iid in file_Justine.ID:
    print(iid.values,file_Justine['NAME'].sel(ID=iid).values,file_Justine['z_perc01'].sel(ID=iid).values)

In [ ]:
kisf = 2
ice_draft.where((file_Justine['Iceshelf_extrap'] == kisf) & (BedMachine_mask_2km['isf_conc'] > 0) & (ice_draft == 0)).plot()

In [ ]:
BedMachine_mask_2km['ocean_conc'].where((file_Justine['Iceshelf_extrap'] == kisf) & (ice_draft == 0) & (BedMachine_mask_2km['isf_conc'] > 0), drop=True).plot()

In [ ]:
((ice_draft == 0) & (BedMachine_mask_2km['isf_conc'] > 0)).plot()